<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#导入相关的包" data-toc-modified-id="导入相关的包-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>导入相关的包</a></span></li><li><span><a href="#导入相关数据" data-toc-modified-id="导入相关数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>导入相关数据</a></span></li><li><span><a href="#使用模型" data-toc-modified-id="使用模型-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>使用模型</a></span><ul class="toc-item"><li><span><a href="#Lasso" data-toc-modified-id="Lasso-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Lasso</a></span></li><li><span><a href="#LassorLar" data-toc-modified-id="LassorLar-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>LassorLar</a></span></li><li><span><a href="#贝叶斯回归" data-toc-modified-id="贝叶斯回归-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>贝叶斯回归</a></span></li><li><span><a href="#ENet" data-toc-modified-id="ENet-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>ENet</a></span></li><li><span><a href="#MLP" data-toc-modified-id="MLP-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>MLP</a></span></li><li><span><a href="#SVR" data-toc-modified-id="SVR-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>SVR</a></span></li><li><span><a href="#Ridge" data-toc-modified-id="Ridge-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Ridge</a></span></li><li><span><a href="#RF" data-toc-modified-id="RF-3.8"><span class="toc-item-num">3.8&nbsp;&nbsp;</span>RF</a></span></li><li><span><a href="#GBoost" data-toc-modified-id="GBoost-3.9"><span class="toc-item-num">3.9&nbsp;&nbsp;</span>GBoost</a></span></li><li><span><a href="#XGBoost" data-toc-modified-id="XGBoost-3.10"><span class="toc-item-num">3.10&nbsp;&nbsp;</span>XGBoost</a></span></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-3.11"><span class="toc-item-num">3.11&nbsp;&nbsp;</span>LightGBM</a></span></li><li><span><a href="#Stacking" data-toc-modified-id="Stacking-3.12"><span class="toc-item-num">3.12&nbsp;&nbsp;</span>Stacking</a></span></li><li><span><a href="#AverageModel" data-toc-modified-id="AverageModel-3.13"><span class="toc-item-num">3.13&nbsp;&nbsp;</span>AverageModel</a></span></li></ul></li></ul></div>

# 导入相关的包

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# 导入相关数据

In [2]:
%time
trains = pd.read_csv('../clean_data/base_train.csv',encoding="gbk")
vals = pd.read_csv('../clean_data/model_fill_train.csv',encoding='gbk')
labels = "blood_sugar"
features = [x for x in list(trains.columns) if x not in ['id','date','blood_sugar','blood_sugar_log']]
features_with_no_cate = [x for x in list(trains.columns) if x not in ['id','date','blood_sugar','blood_sugar_log','gender','age']]
x_val, x_test, y_val, y_test = train_test_split(vals[features], vals[labels],test_size=0.1, random_state=42)
def re(y):
    return np.exp(np.exp(np.exp(-y)))

Wall time: 0 ns


In [3]:
first = ["feature_34","feature_35","feature_36",'feature_7','feature_26',
         'feature_30','feature_29','feature_31', 'feature_28','feature_22',]
second = ["feature_34","feature_35","feature_36",'feature_7','feature_25']
third = ["feature_34","feature_35","feature_36",'feature_7','feature_25',
         'feature_30','feature_29','feature_31', 'feature_28','feature_22']
for i in second:
    features_with_no_cate.remove(i)

In [20]:
features_with_no_cate

['feature_0',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_8',
 'feature_9',
 'feature_10',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_20',
 'feature_21',
 'feature_22',
 'feature_23',
 'feature_24',
 'feature_26',
 'feature_27',
 'feature_28',
 'feature_29',
 'feature_30',
 'feature_31',
 'feature_32',
 'feature_33']

# 使用模型

In [4]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingRegressor
import xgboost as xgb
import lightgbm as lgb

In [35]:
# vals = pd.read_csv('../clean_data/model_fill_train.csv',encoding='gbk')
# trains = pd.concat([trains, vals],axis=0,ignore_index=True)
# train = trains[features_with_no_cate]
# y_train = trains[labels]
# test = x_test[features_with_no_cate]
# y_test = y_test

In [7]:
train = pd.read_csv("../raw_data/fea_train.csv")
test = pd.read_csv("../raw_data/fea_test.csv")
labels = "blood_sugar"
y_train = train[labels]
second = ["feature_34","feature_35","feature_36",'feature_7','feature_25']
features_with_no_cate = list(test.columns)
for i in second:
    features_with_no_cate.remove(i)
train = train[features_with_no_cate]
test = test[features_with_no_cate]

KeyError: 'blood_sugar'

In [140]:
train.shape
test.shape
y_train
train.columns
test.columns

(5606, 71)

(1000, 71)

0       6.06
1       5.39
2       5.59
3       4.30
4       5.97
5       5.11
6       5.94
7       5.66
8       5.48
9       4.44
10      5.48
11      5.32
12      5.08
13      4.63
14      4.92
15      7.08
16      4.66
17      6.58
18      4.29
19      4.55
20      5.16
21      5.13
22      4.37
23      5.17
24      4.64
25      5.82
26      6.29
27      5.10
28      5.41
29      5.48
        ... 
5576    4.99
5577    4.83
5578    4.81
5579    5.12
5580    5.41
5581    7.51
5582    6.43
5583    5.01
5584    6.55
5585    4.81
5586    5.29
5587    4.71
5588    4.74
5589    4.76
5590    4.48
5591    4.75
5592    4.69
5593    5.21
5594    5.28
5595    4.78
5596    4.75
5597    5.19
5598    4.38
5599    5.09
5600    5.05
5601    5.17
5602    5.20
5603    5.63
5604    4.84
5605    5.40
Name: blood_sugar, Length: 5606, dtype: float64

Index(['age', 'feature_0', 'feature_1', 'feature_10', 'feature_11',
       'feature_12', 'feature_13', 'feature_14', 'feature_2', 'feature_20',
       'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_26',
       'feature_27', 'feature_28', 'feature_29', 'feature_3', 'feature_30',
       'feature_31', 'feature_32', 'feature_33', 'feature_4', 'feature_5',
       'feature_6', 'feature_8', 'feature_9', 'feature_0_is_normal',
       'feature_1_is_normal', 'feature_1_feature_0',
       'feature_1_feature_0_is_normal', 'feature_2_is_normal',
       'feature_3_is_normal', 'feature_4_is_normal', 'feature_5_is_normal',
       'feature_6_is_normal', 'feature_8_is_normal', 'feature_9_is_normal',
       'feature_10_is_normal', 'feature_10_gender_is_normal',
       'feature_11_is_normal', 'feature_10_feature_11', 'feature_12_is_normal',
       'feature_13_is_normal', 'feature_14_is_normal',
       'feature_14_gender_is_normal', 'feature_20_is_normal',
       'feature_21_is_normal', '

Index(['age', 'feature_0', 'feature_1', 'feature_10', 'feature_11',
       'feature_12', 'feature_13', 'feature_14', 'feature_2', 'feature_20',
       'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_26',
       'feature_27', 'feature_28', 'feature_29', 'feature_3', 'feature_30',
       'feature_31', 'feature_32', 'feature_33', 'feature_4', 'feature_5',
       'feature_6', 'feature_8', 'feature_9', 'feature_0_is_normal',
       'feature_1_is_normal', 'feature_1_feature_0',
       'feature_1_feature_0_is_normal', 'feature_2_is_normal',
       'feature_3_is_normal', 'feature_4_is_normal', 'feature_5_is_normal',
       'feature_6_is_normal', 'feature_8_is_normal', 'feature_9_is_normal',
       'feature_10_is_normal', 'feature_10_gender_is_normal',
       'feature_11_is_normal', 'feature_10_feature_11', 'feature_12_is_normal',
       'feature_13_is_normal', 'feature_14_is_normal',
       'feature_14_gender_is_normal', 'feature_20_is_normal',
       'feature_21_is_normal', '

In [37]:
# sub_test = pd.read_csv('../clean_data/test.csv',encoding='gbk')
# test_sub = sub_test[features_with_no_cate]

In [38]:
test_sub.head()
test_sub.shape

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_8,feature_9,feature_10,...,feature_23,feature_24,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33
0,23.850,26.690,116.080,34.360,82.750,46.030,36.720,1.070,5.030,1.440,...,0.453,91.7,333.0,12.3,241.0,10.8,12.8,0.26,58.4,33.2
1,29.750,34.980,90.070,111.430,71.900,44.090,27.810,2.480,5.510,1.220,...,0.518,88.7,330.0,12.0,242.0,11.5,14.6,0.28,59.3,29.3
2,31.187,33.966,92.145,43.766,78.465,47.218,30.994,1.992,5.381,1.301,...,0.506,88.5,338.0,12.1,398.0,8.9,9.9,0.35,50.2,40.1
3,17.980,16.630,95.950,23.410,78.160,45.440,32.720,5.940,6.800,1.380,...,0.420,91.1,329.0,12.6,247.0,11.6,13.9,0.29,53.7,38.0
4,19.120,19.800,76.970,15.700,80.760,46.900,33.860,0.740,6.230,2.250,...,0.440,95.2,334.0,12.2,335.0,10.4,11.9,0.35,52.0,39.4


(1000, 27)

In [39]:
# gender_onehot_train = pd.get_dummies(trains['gender'])
# gender_onehot_train.rename(columns={0:'gender_0',1:'gender_1'},inplace=True)
# train = pd.concat([train, gender_onehot_train, trains['age']],axis=1)
# gender_onehot_test = pd.get_dummies(x_test['gender'])
# gender_onehot_test.rename(columns={0:'gender_0',1:'gender_1'},inplace=True)
# test = pd.concat([test, gender_onehot_test, x_test['age']],axis=1)
# gender_onehot_test = pd.get_dummies(sub_test['gender'])
# gender_onehot_test.rename(columns={0:'gender_0',1:'gender_1'},inplace=True)
# test_sub = pd.concat([test_sub, gender_onehot_test, sub_test['age']],axis=1)

In [40]:
# all_data = pd.concat([train, test],axis=0,ignore_index=True)
# age_data = pd.get_dummies(all_data['age'])
# all_data = pd.concat([all_data, age_data],axis=1)
# all_data.drop('age',inplace=True,axis=1)
# train = all_data[:train.shape[0]]
# test = all_data[train.shape[0]:]

In [82]:
train.shape
test.shape

(5606, 76)

(1000, 76)

In [5]:
#Validation function
n_folds = 10

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)
def rmsle(y,y_pred):
#     return 0.5 * mean_squared_error(re(y), re(y_pred))
    return 0.5 * mean_squared_error(y, y_pred)

## Lasso

In [6]:
%time
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=42))
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
lasso.fit(train,y_train)
lasso_train_pred = lasso.predict(train)
lasso_pred = lasso.predict(test)
print("train: ",rmsle(y_train, lasso_train_pred))
# print("test: ", rmsle(y_test,lasso_pred))

Wall time: 0 ns


NameError: name 'train' is not defined

## LassorLar

In [80]:
%time
from sklearn import linear_model
LassorLar = linear_model.LassoLars(alpha=.1)
score = rmsle_cv(LassorLar)
print("LassorLar score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
LassorLar.fit(train,y_train)
LassorLar_train_pred = LassorLar.predict(train)
LassorLar_pred = LassorLar.predict(test)
print("train: ",rmsle(y_train, LassorLar_train_pred))
# print("test: ", rmsle(y_test,LassorLar_pred))

Wall time: 0 ns
LassorLar score: 1.2479 (0.0632)



LassoLars(alpha=0.1, copy_X=True, eps=2.2204460492503131e-16,
     fit_intercept=True, fit_path=True, max_iter=500, normalize=True,
     positive=False, precompute='auto', verbose=False)

train:  0.778154696431


## 贝叶斯回归

In [84]:
%time
# 贝叶斯回归
from sklearn import linear_model
Bayreg = linear_model.BayesianRidge()
score = rmsle_cv(Bayreg)
print("Bayreg score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
Bayreg.fit(train,y_train)
Bayreg_train_pred = Bayreg.predict(train)
Bayreg_pred = Bayreg.predict(test)
print("train: ",rmsle(y_train, Bayreg_train_pred))
# print("test: ", rmsle(y_test,Bayreg_pred))

Wall time: 0 ns
Bayreg score: 1.1316 (0.0533)



BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)

train:  0.625925582096


## ENet

In [85]:
%time
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.005, l1_ratio=.9, random_state=42))
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
ENet.fit(train,y_train)
ENet_train_pred = ENet.predict(train)
ENet_pred = ENet.predict(test)
print("train: ",rmsle(y_train, ENet_train_pred))
# print("test: ", rmsle(y_test,ENet_pred))

Wall time: 0 ns
ElasticNet score: 1.1348 (0.0554)



Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('elasticnet', ElasticNet(alpha=0.005, copy_X=True, fit_intercept=True, l1_ratio=0.9,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=42, selection='cyclic', tol=0.0001, warm_start=False))])

train:  0.625277792846


## MLP

In [87]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
mlp =  make_pipeline(StandardScaler(), MLPRegressor(random_state=42,learning_rate ="adaptive",max_iter=500))
score = rmsle_cv(mlp)
print("mlp score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
mlp.fit(train,y_train)
mlp_train_pred = mlp.predict(train)
mlp_pred = mlp.predict(test)
print("train: ",rmsle(y_train, mlp_train_pred))
# print("test: ", rmsle(y_test,mlp_pred))

mlp score: 1.3107 (0.0882)



Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlpregressor', MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='adaptive',
   ...       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False))])

train:  0.476601833956


## SVR

In [88]:
%time
from sklearn import svm
svr = svm.SVR()
score = rmsle_cv(svr)
print("svr score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
svr.fit(train,y_train)
svr_train_pred = svr.predict(train)
svr_pred = svr.predict(test)
print("train: ",rmsle(y_train, svr_train_pred))
# print("test: ", rmsle(y_test,svr_pred))

Wall time: 0 ns
svr score: 1.2605 (0.0766)



SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

train:  0.411049205205


## Ridge

In [89]:
%time
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
KRR.fit(train,y_train)
KRR_train_pred = KRR.predict(train)
KRR_pred = KRR.predict(test)
print("train: ",rmsle(y_train, KRR_train_pred))
# print("test: ", rmsle(y_test,KRR_pred))

Wall time: 0 ns
Kernel Ridge score: 7.1461 (11.0879)



KernelRidge(alpha=0.6, coef0=2.5, degree=2, gamma=None, kernel='polynomial',
      kernel_params=None)

train:  0.416427977505


ValueError: Found input variables with inconsistent numbers of samples: [170, 1000]

## RF

In [90]:
%time
rf = RandomForestRegressor(random_state=530)
score = rmsle_cv(rf)
print("\r rf score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
rf.fit(train,y_train)
rf_train_pred = rf.predict(train)
rf_pred = rf.predict(test)
print("train: ",rmsle(y_train, rf_train_pred))
# print("test: ", rmsle(y_test,rf_pred))

Wall time: 0 ns
 rf score: 1.1919 (0.0707)



RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=530, verbose=0, warm_start=False)

train:  0.133344048609


## GBoost

In [91]:
%time
GBoost = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
GBoost.fit(train, y_train)
GBoost_train_pred = GBoost.predict(train)
GBoost_pred = GBoost.predict(test)
print("train: ",rmsle(y_train, GBoost_train_pred))
# print("test: ", rmsle(y_test,GBoost_pred))

Wall time: 0 ns
Gradient Boosting score: 1.1441 (0.0528)



GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='huber', max_depth=4,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=300,
             presort='auto', random_state=5, subsample=1.0, verbose=0,
             warm_start=False)

train:  0.537498324463


ValueError: Found input variables with inconsistent numbers of samples: [170, 1000]

## XGBoost

In [144]:
%time
x_val, x_test, y_val, y_test = train_test_split(train[features_with_no_cate], y_train,test_size=0.1, random_state=42)
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.01, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=500,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             seed =1024, nthread = -1)

score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
model_xgb.fit(x_val, y_val,eval_set=[(x_test, y_test)])
xgb_train_pred = model_xgb.predict(train)
xgb_pred = model_xgb.predict(test)
print("train: ",rmsle(y_train, xgb_train_pred))
# print("test: ", rmsle(y_test,xgb_pred))

Wall time: 0 ns
Xgboost score: 1.1252 (0.0513)

[0]	validation_0-rmse:5.1742
[1]	validation_0-rmse:5.12572
[2]	validation_0-rmse:5.07726
[3]	validation_0-rmse:5.0294
[4]	validation_0-rmse:4.98203
[5]	validation_0-rmse:4.9351
[6]	validation_0-rmse:4.88863
[7]	validation_0-rmse:4.84269
[8]	validation_0-rmse:4.79769
[9]	validation_0-rmse:4.7527
[10]	validation_0-rmse:4.70795
[11]	validation_0-rmse:4.66406
[12]	validation_0-rmse:4.62077
[13]	validation_0-rmse:4.57761
[14]	validation_0-rmse:4.53509
[15]	validation_0-rmse:4.49265
[16]	validation_0-rmse:4.45089
[17]	validation_0-rmse:4.40939
[18]	validation_0-rmse:4.36882
[19]	validation_0-rmse:4.32809
[20]	validation_0-rmse:4.28787
[21]	validation_0-rmse:4.24902
[22]	validation_0-rmse:4.20962
[23]	validation_0-rmse:4.17061
[24]	validation_0-rmse:4.13241
[25]	validation_0-rmse:4.09461
[26]	validation_0-rmse:4.05671
[27]	validation_0-rmse:4.01963
[28]	validation_0-rmse:3.98247
[29]	validation_0-rmse:3.94604
[30]	validation_0-rmse:3.91019
[31]	

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.4603,
       gamma=0.0468, learning_rate=0.01, max_delta_step=0, max_depth=3,
       min_child_weight=1.7817, missing=None, n_estimators=500, nthread=-1,
       objective='reg:linear', reg_alpha=0.464, reg_lambda=0.8571,
       scale_pos_weight=1, seed=1024, silent=1, subsample=0.5213)

train:  0.53480265664


## LightGBM

In [155]:
%time
cate_fea =  [ 'feature_0_is_normal',
       'feature_1_is_normal', 'feature_1_feature_0',
       'feature_1_feature_0_is_normal', 'feature_2_is_normal',
       'feature_3_is_normal', 'feature_4_is_normal', 'feature_5_is_normal',
       'feature_6_is_normal', 'feature_8_is_normal', 'feature_9_is_normal',
       'feature_10_is_normal', 'feature_10_gender_is_normal',
       'feature_11_is_normal', 'feature_10_feature_11', 'feature_12_is_normal',
       'feature_13_is_normal', 'feature_14_is_normal',
       'feature_14_gender_is_normal', 'feature_20_is_normal',
       'feature_21_is_normal', 'feature_22_is_normal', 'feature_23_is_normal',
       'feature_24_is_normal', 'feature_25_is_normal', 'feature_26_is_normal',
       'feature_27_is_normal', 'feature_28_is_normal', 'feature_29_is_normal',
       'feature_30_is_normal', 'feature_31_is_normal', 'feature_32_is_normal',
       'feature_20_feature_32', 'feature_33_is_normal',
       'feature_20_feature_33', 'feature_34_is_normal',
       'feature_20_feature_34', 'feature_35_is_normal',
       'feature_20_feature_35', 'feature_36_is_normal',
       'feature_20_feature_36', 'gender_0', 'gender_1']
def evalerror(pred, df):
    label = df.get_label().values.copy()
    score = mean_squared_error(label,pred)*0.5
    return ('mse',score,False)
x_val, x_test, y_val, y_test = train_test_split(train[features_with_no_cate], y_train,test_size=0.1, random_state=42)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=550,
                              max_bin = 25, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =7, min_sum_hessian_in_leaf = 12)
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
model_lgb.fit(train, y_train,eval_set=[(x_test,y_test)],,categorical_feature=cate_fea)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = model_lgb.predict(test.values)
print("train: ",rmsle(y_train, lgb_train_pred))
sub_pd = pd.DataFrame(lgb_pred)
sub_pd.describe()
# sub_pd.to_csv('../submission/sub_average_7_2_h.csv',header=False, index=False)
# print("test: ",rmsle(y_test, lgb_pred))

Wall time: 0 ns
LGBM score: 1.1173 (0.0797)

[1]	valid_0's l2: 1.55178
[2]	valid_0's l2: 1.53786
[3]	valid_0's l2: 1.51762
[4]	valid_0's l2: 1.50622
[5]	valid_0's l2: 1.48996
[6]	valid_0's l2: 1.48129
[7]	valid_0's l2: 1.4724
[8]	valid_0's l2: 1.46608
[9]	valid_0's l2: 1.45999
[10]	valid_0's l2: 1.4513
[11]	valid_0's l2: 1.44622
[12]	valid_0's l2: 1.42977
[13]	valid_0's l2: 1.42303
[14]	valid_0's l2: 1.41684
[15]	valid_0's l2: 1.40458
[16]	valid_0's l2: 1.39575
[17]	valid_0's l2: 1.39106
[18]	valid_0's l2: 1.3856
[19]	valid_0's l2: 1.37997
[20]	valid_0's l2: 1.37117
[21]	valid_0's l2: 1.36182
[22]	valid_0's l2: 1.34874
[23]	valid_0's l2: 1.3434
[24]	valid_0's l2: 1.33827
[25]	valid_0's l2: 1.33248
[26]	valid_0's l2: 1.33001
[27]	valid_0's l2: 1.32037
[28]	valid_0's l2: 1.31681
[29]	valid_0's l2: 1.30819
[30]	valid_0's l2: 1.30178
[31]	valid_0's l2: 1.29407
[32]	valid_0's l2: 1.28917
[33]	valid_0's l2: 1.28638
[34]	valid_0's l2: 1.28185
[35]	valid_0's l2: 1.27569
[36]	valid_0's l2: 1.27

LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, bagging_seed=9,
       boosting_type='gbdt', colsample_bytree=1, feature_fraction=0.2319,
       feature_fraction_seed=9, learning_rate=0.05, max_bin=25,
       max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_data_in_leaf=7, min_split_gain=0, min_sum_hessian_in_leaf=12,
       n_estimators=550, nthread=-1, num_leaves=5, objective='regression',
       reg_alpha=0, reg_lambda=0, seed=0, silent=True, subsample=1,
       subsample_for_bin=50000, subsample_freq=1)

train:  0.436551454971


,0
count,1000.000000
mean,5.614748
std,0.608677
min,4.643751
25%,5.170262
50%,5.507668
75%,5.914453
max,10.125190


## Stacking

In [136]:
regressors = [lasso, ENet, GBoost]
stregr = StackingRegressor(regressors=regressors, meta_regressor=model_xgb)
score = rmsle_cv(stregr)
print("stacking score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))
stregr.fit(train, y_train)
stregr_train_pred = stregr.predict(train)
stregr_pred = stregr.predict(test)
print("train: ",rmsle(y_train, stregr_train_pred))
# print("test: ", rmsle(y_test,stregr_pred))

stacking score: 1.1304 (0.0672)



StackingRegressor(meta_regressor=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.4603,
       gamma=0.0468, learning_rate=0.01, max_delta_step=0, max_depth=3,
       min_child_weight=1.7817, missing=None, n_estimators=500, nthread=-1,
       objective='reg:linear', reg_alpha=0.464, reg_lambda=0.8571,
       scale_pos_weight=1, seed=1024, silent=1, subsample=0.5213),
         regressors=[Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('lasso', Lasso(alpha=0.0005, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, ran...           presort='auto', random_state=5, subsample=1.0, verbose=0,
             warm_start=False)],
         verbose=0)

train:  0.418254426919


## AverageModel

In [96]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [135]:
%time
averaged_models = AveragingModels(models = (model_lgb, rf, stregr))
score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
averaged_models.fit(train,y_train)
ave_train_pred = averaged_models.predict(train)
ave_pred = averaged_models.predict(test)
print("train: ",rmsle(y_train, ave_train_pred))
sub_pd = pd.DataFrame(ave_pred)
# sub_pd.to_csv('../submission/sub_average_7_2_c.csv',header=False, index=False)
# print("test: ", rmsle(y_test,ave_pred))

Wall time: 0 ns
 Averaged base models score: 1.1165 (0.0606)



AveragingModels(models=(LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, bagging_seed=9,
       boosting_type='gbdt', colsample_bytree=1, feature_fraction=0.2319,
       feature_fraction_seed=9, learning_rate=0.05, max_bin=25,
       max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_data_in_leaf...to', random_state=5, subsample=1.0, verbose=0,
             warm_start=False)],
         verbose=0)))

train:  0.281745426925


In [60]:
sub = averaged_models.predict(test_sub)
sub_pd = pd.DataFrame(sub)
sub_pd.to_csv('../submission/sub_average_7_1_c.csv',header=False, index=False)

ValueError: Number of features of the model must match the input. Model n_features is 95 and input n_features is 30 

In [ ]:
sub_pd.describe()

In [ ]:
print('RMSLE score on train data:')
print(rmsle(y_train,xgb_train_pred*0.70 +
               lgb_train_pred*0.15 + ave_train_pred*0.15 ))
print('RMSLE score on test data:')
print(rmsle(y_test,ave_pred*0.65 +
               lgb_pred*0.10 + xgb_pred*0.25 ))